# Рекомендательные системы

## Урок 3. Коллаборативная фильтрация

### Подготовка к проекту. Постановка задачи

*Примечание: пока не сделал, надеюсь вернуться к вопросам/ гипотезам позже. Пока просто констатирую постановку задачи.*

Задача:
- создать и внедрить рекомендательную систему для рассылок клиентам
- цель: росты выручки на 6% за 4 месяца кампании, рост retention на 3% и среднего чека на 3%
- рассылки будут получать 5% клиентов через email, 20% клиентов через push-уведомления, все оффлайн клиенты на чеке
- в рассылке будет 5 товаров: 3 акционных, 1 новый, 1 для роста среднего чека

    

### Практическая часть. ALS Grid Search

In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import bsr_matrix

from implicit.als import AlternatingLeastSquares

from sklearn.model_selection import ParameterGrid

In [2]:
from tqdm.notebook import tqdm

In [3]:
# Precision@K
def precision_at_k(recommended_list, bought_list, k=5):
    try:
        _rec_list = recommended_list[:k]
        _b_and_r = np.intersect1d(bought_list, _rec_list)
        return _b_and_r.size / len(_rec_list)
    except (ZeroDivisionError, TypeError):
        return 0.0

def mean_precision_at_k(df, rec, bought, k=5):
    _result = df.apply(
        lambda row: precision_at_k(row[rec], row[bought], k),
        axis=1
    )
    return np.mean(_result)

In [4]:
data = pd.read_csv('transaction_data_filtered.csv.zip')
data.shape

(2595732, 5)

In [5]:
# train-test split
test_size_weeks = 3
data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]

In [6]:
# Actual test items
actual = data_test.groupby('user_id').agg(actual=('item_id', list))
actual.shape

(1991, 1)

In [7]:
%%time
# Матрица USER/ITEM
# Оставим только транзакции с товарами, входящими в Top-5000
top_5000 = data_train.groupby('item_id').agg(n_sold=('quantity', 'sum')).nlargest(5000, 'n_sold')

user_item_df = (
    data_train[data_train['item_id'].isin(top_5000.index)]
    .groupby(['user_id', 'item_id'])
    .agg({'quantity': 'count'})
    .reset_index()
)

# Мапинг user_id / item_id -> индекс строки / столбца матрицы и обратно
def get_index_maps(series):
    _direct = dict(enumerate(series.unique()))
    _reverse = {id: i for i, id in _direct.items()}
    return _direct, _reverse

to_uid, from_uid = get_index_maps(user_item_df['user_id'])
to_iid, from_iid = get_index_maps(user_item_df['item_id'])
iid_array = np.array(list(to_iid.values()))

user_item_matrix = bsr_matrix(
    (user_item_df['quantity'].astype(float),  # data
     (user_item_df['user_id'].map(from_uid),  # row
      user_item_df['item_id'].map(from_iid))),  # col
    shape=(len(from_uid), len(from_iid)))

sparse_user_item = user_item_matrix.tocsr()
sparse_item_user = user_item_matrix.T.tocsr()

Wall time: 994 ms


In [8]:
%%time
# ALS Grid Search
als_param_grid = {
    'factors': [2, 3, 4, 8, 16, 32],
    'regularization': [0, 0.01, 0.05],
    'iterations': [10, 15, 20]
}
grid_len = np.prod([len(v) for v in als_param_grid.values()])

result = actual

with tqdm(desc="ALS Grid Search", total=grid_len) as progress:
    for prm in ParameterGrid(als_param_grid):
        model_name = (
            f"AlternatingLeastSquares(factors={prm['factors']}, "
            f"regularization={prm['regularization']}, "
            f"iterations={prm['iterations']})"
        )
        model = AlternatingLeastSquares(**prm)
        model.fit(sparse_item_user, show_progress=False)
        fast_recs = model.user_factors @ model.item_factors.T
        rec_matrix = iid_array[np.argsort(-fast_recs)[:, :5]]
        rec_df = pd.DataFrame.from_records(
            np.expand_dims(rec_matrix, axis=1),
            columns = [model_name],
            index = from_uid
        )
        result = result.join(rec_df)
        progress.update(1)

ALS Grid Search:   0%|          | 0/54 [00:00<?, ?it/s]

Wall time: 4min 48s


In [9]:
%%time
# Выбираем лучшие параметры по precision@5
precision_df = pd.DataFrame(
    ((model_name, mean_precision_at_k(result, model_name, 'actual'))
     for model_name in result.columns[1:]),
    columns=['model', 'precision@5']
)

Wall time: 13.9 s


In [10]:
(
    precision_df
    .sort_values('precision@5', ascending=False)
    .head(10)
    .style.set_properties(subset=['model'], **{'width': '450px'})
)

,model,precision@5
9,"AlternatingLeastSquares(factors=3, regularization=0, iterations=10)",0.202612
12,"AlternatingLeastSquares(factors=3, regularization=0, iterations=15)",0.202310
15,"AlternatingLeastSquares(factors=3, regularization=0, iterations=20)",0.201507
14,"AlternatingLeastSquares(factors=3, regularization=0.05, iterations=15)",0.201406
47,"AlternatingLeastSquares(factors=32, regularization=0.05, iterations=10)",0.201005
17,"AlternatingLeastSquares(factors=3, regularization=0.05, iterations=20)",0.200703
23,"AlternatingLeastSquares(factors=4, regularization=0.05, iterations=15)",0.200100
21,"AlternatingLeastSquares(factors=4, regularization=0, iterations=15)",0.200000
22,"AlternatingLeastSquares(factors=4, regularization=0.01, iterations=15)",0.199297
16,"AlternatingLeastSquares(factors=3, regularization=0.01, iterations=20)",0.198895
